In [1]:
import csv
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import cloudscraper
from datetime import datetime
#from msedge.selenium_tools import Edge, EdgeOptions
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium import webdriver
from selenium.webdriver.edge import service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

In [2]:
temp = 'https://ca.indeed.com/jobs?q={}&l={}'

In [3]:
def get_url(job_position, location):
    """Generate an indeed job posting URL from position and Location"""
    temp = 'https://ca.indeed.com/jobs?q={}&l={}'
    position = job_position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = temp.format(position, location)
    return url

In [4]:
url = get_url('Data Analyst', 'Toronto ON')
url

'https://ca.indeed.com/jobs?q=Data+Analyst&l=Toronto+ON'

In [5]:
# Extract each job posting record
def job_posting_record(card):
    Job_title = card.find_element(By.CLASS_NAME, 'jobTitle').text
    Company = card.find_element(By.CLASS_NAME, 'companyName').text
    try:
        Rating = card.find_element(By.CLASS_NAME, 'ratingNumber').text
    except NoSuchElementException:
        Rating = 'N/A'
    Location = card.find_element(By.CLASS_NAME, 'companyLocation').text
    try:
        Salary = card.find_element(By.CSS_SELECTOR, 'div.metadata.salary-snippet-container > div.attribute_snippet').text
    except NoSuchElementException:
        Salary = 'N/A'
    Date_Posted = card.find_element(By.XPATH, '//span[@class="date"]').text.split("\n")[-1]
    Date_Extracted = datetime.today().strftime('%Y-%m-%d')
    try:
        Easy_Apply = card.find_element(By.CLASS_NAME, 'ialbl.iaTextBlack').text
    except NoSuchElementException:
        Easy_Apply = 'No'
    Job_summary = card.find_element(By.CLASS_NAME, 'job-snippet').text
    Job_url = card.find_element(By.XPATH, '//h2[contains(@class, "jobTitle")]/a[@href]').get_attribute('href')
    return (Job_title, Company, Rating, Location, Salary, Date_Posted, Date_Extracted, Easy_Apply, Job_summary, Job_url)

In [6]:
# Extract all the cards from each page
def posting_page_record(posting, job_list, url_set):
    for card in posting:
        record = job_posting_record(card)
        # ensure no duplicates are added
        #if record[-1] not in url_set:
        job_list.append(record)
        url_set.add(record[-1])

In [10]:
def postings(job_position, location):
    scrapped_jobs = []
    scrapped_urls = set()
    
    url = get_url(job_position, location)
    
    # web driver
    chromeOptions = ChromeOptions()
    chromeOptions.add_argument("start-maximized")
    chromeOptions.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36')
    s=ChromeService(r'C:\Users\Administrator\Desktop\chromedriver.exe') # <-- Set the correct path to your ChromeDriver
    driver = webdriver.Chrome(service=s, options=chromeOptions)
    driver.maximize_window()
    driver.set_page_load_timeout(10)
    driver.get(url)
    
    time.sleep(20)
    
    # extract jobs data
    while True:
        posting = driver.find_elements(By.CLASS_NAME, "job_seen_beacon")
        posting_page_record(posting, scrapped_jobs, scrapped_urls)
        try:
            time.sleep(20)
            #WebDriverWait(driver, 30).until(EC.invisibility_of_element_located((By.XPATH, '//div[@class="gnav-CookiePrivacyNoticeBanner"]')))
            #WebDriverWait(driver, 20).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.gnav-CookiePrivacyNoticeBanner')))
            #WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//a[@aria-label="Next Page"]'))).click()
            #WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[aria-label=Next Page]'))).click()
            next_page = driver.find_element(By.XPATH, '//a[@aria-label="Next Page"]')
            driver.execute_script("arguments[0].click();", next_page)
            #ActionChains(driver).move_to_element(next_page).click().perform()
        except NoSuchElementException:
            break
        #except TimeoutException:
           # break
        except ElementNotInteractableException:
            driver.find_element(By.XPATH, '//button[@aria-label="close"]').click()
            posting_page_record(posting, scrapped_jobs, scrapped_urls)
            continue
    
    #close driver
    driver.close()
    #save data
    save_to_file(scrapped_jobs)

In [11]:
def save_to_file(records):
    with open('DA_Postings_31JUL2023.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job_title', 'Company', 'Rating', 'Location', 'Salary', 'Date_Posted', 'Date_Extracted', 'Easy_Apply', 'Job_Summary', 'Job_url'])
        writer.writerows(records)

In [12]:
postings('Data Analyst', 'Toronto ON')